In [1]:
import sys
import numpy as np
import random as rd
import lakiaro3 as lk
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout,QMainWindow
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from PyQt5 import uic
import cv2
import time
form_class = uic.loadUiType("lakiaro.ui")[0]

class MyApp(QMainWindow,form_class):

    def __init__(self):
        super().__init__()
        self.setupUi(self)
         
        #load image
        self.load_img()

        self.pushButton.clicked.connect(self.btn_clicked)
        self.ingame = 0 # 게임 진행중 여부
    
    def load_img(self):

        
        gravel = cv2.imread('image/gravel.png')
        self.gravel = cv2.cvtColor(gravel, cv2.COLOR_BGR2RGB)
        
        dirt = cv2.imread('image/dirt.png')
        self.dirt = cv2.cvtColor(dirt, cv2.COLOR_BGR2RGB)
        
        #아래시작 위/오른쪽끝
        root_thick_1 = cv2.imread('image/root_thick_1.png')
        self.root_thick_1 = cv2.cvtColor(root_thick_1, cv2.COLOR_BGR2RGB)

        root_thick_2 = cv2.imread('image/root_thick_2.png')
        self.root_thick_2 = cv2.cvtColor(root_thick_2, cv2.COLOR_BGR2RGB)
        
        root_turn_1 = cv2.imread('image/root_turn_1.png')
        self.root_turn_1 = cv2.cvtColor(root_turn_1, cv2.COLOR_BGR2RGB)
        
        root_turn_2 = cv2.imread('image/root_turn_2.png')
        self.root_turn_2 = cv2.cvtColor(root_turn_2, cv2.COLOR_BGR2RGB)  
        
        root_thin_1 = cv2.imread('image/root_thin_1.png')
        self.root_thin_1 = cv2.cvtColor(root_thin_1, cv2.COLOR_BGR2RGB)
        
        root_thin_2 = cv2.imread('image/root_thin_2.png')
        self.root_thin_2 = cv2.cvtColor(root_thin_2, cv2.COLOR_BGR2RGB)
        
        root_last = cv2.imread('image/root_last.png')
        self.root_last = cv2.cvtColor(root_last, cv2.COLOR_BGR2RGB)
        
    def btn_clicked(self):
        self.ingame = 1 #game start
        self.game_init_1()
        
    def game_init_1(self):
        
        # game setting
        self.laki = lk.lakiaro(0.15) #자갈 난이도 
        self.hoe_level = 8 #호미 난이도 
        
        # create game, print game info
        board_cv2 = cv2.imread('image/lariaro_init.png')
        self.board_cv2 = cv2.cvtColor(board_cv2, cv2.COLOR_BGR2RGB)
        
        light_dic = {5:18, 6:20, 7:22, 8:25, 9:28} #장광고유동
        self.light_dig = light_dic[self.hoe_level] #얕게파기 횟수
        self.ans_df, self.total_cnt_root, self.total_cnt_dirt, self.total_cnt_gravel = self.laki.create_all() 
        self.m_df = self.laki.create_frame(2) 
        print(self.ans_df,self.total_cnt_root, self.total_cnt_dirt, self.total_cnt_gravel)
        #set interface
        self.label_7.setText(str(self.total_cnt_root))
        self.label_8.setText(str(self.total_cnt_dirt))
        self.label_9.setText(str(self.total_cnt_gravel))
        self.label_10.setText(str(self.light_dig))
        self.progressBar.setValue(100)
        
        # print init
        height, width, channel = self.board_cv2.shape
        bytesPerLine = 3 * width
        qImg_board = QImage(self.board_cv2.data, width, height, bytesPerLine, QImage.Format_RGB888)
        self.label_img.setPixmap(QPixmap(qImg_board))        
        
        # create answer pic
        ans_board_cv2 = cv2.imread('image/lariaro_init.png')
        self.ans_board_cv2 = cv2.cvtColor(ans_board_cv2, cv2.COLOR_BGR2RGB)
        step_size = 30
        angle_list= [0,90,180,270]
        for i in range(12):
            for j in range(12):
                val = self.ans_df.iloc[(i,j)]
                pic_size = 30
                if val == 0 : #흙이면
                    pic = self.dirt
                    self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] =  self.rotate_img(pic)
                elif val == 91 : #자갈이면
                    pic = self.gravel
                    self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic)          
                elif val == 99 : #꽃
                    pass
                
                elif val >10 and val < 90: #뿌리면
                    
                    #initial info
                    around_lst = lk.input_info.xy(i+1,j+1) #현재좌표 포함 9좌표
                    first = 0
                    last = 0
                    temp = [] 
                    #print(i,j,around_lst)
                    for t in range(len(around_lst)):
                        loc = (around_lst[t][0],around_lst[t][1])
                        if self.ans_df.iloc[loc] == val +1:
                            next_loc = loc # 다음뿌리 위치
                            temp.append(1)
                        if self.ans_df.iloc[loc] == val -1:
                            prio_loc = loc # 이전뿌리 위치
                            temp.append(2)
                    if len(temp)<2: #첫번째, 마지막 구분
                        if temp[0] == 1:
                            first =1
                        elif temp[0] == 2:
                            last =1
                            
                    #뿌리로 사진 배정
                    if val % 10 < 5 : #굵은뿌리                   
                        pic1 = self.root_thick_1
                        pic2 = self.root_thick_2
                        pass
                    elif val % 10  == 5: #얇아지는 뿌리
                        pic1 = self.root_turn_1
                        pic2 = self.root_turn_2
                    elif val % 10 > 5 : #얇은 뿌리
                        if last == 0:
                            pic1 = self.root_thin_1
                            pic2 = self.root_thin_2
                        if last ==1:
                            pic1 = self.root_last
                            
                    if first ==1: #첫번째뿌리
                        nx_diff = next_loc[0] - i  
                        ny_diff = next_loc[1] - j        
                        if ny_diff == 0 and nx_diff != 0:
                            self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,1)
                        else:
                            self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,2)
                    elif last ==1: #마지막뿌리
                        px_diff = prio_loc[0] - i  
                        py_diff = prio_loc[1] - j 
                        if py_diff == 0 and px_diff != 0:
                            if px_diff<0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,3)
                            elif px_diff>0:   
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,1)
                        elif py_diff != 0 and px_diff == 0:
                            if py_diff<0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,2)
                            elif py_diff>0: 
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,4)
                    else:
                        nx_diff = next_loc[0] - i  
                        ny_diff = next_loc[1] - j        
                        px_diff = prio_loc[0] - i  
                        py_diff = prio_loc[1] - j 

                        #방향별 구분
                        if ny_diff == 0 and py_diff != 0:
                            if nx_diff < 0 and py_diff < 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(self.rotate_img(pic2,2),6)
                            elif nx_diff > 0 and py_diff < 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,2)
                            elif nx_diff > 0 and py_diff > 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(self.rotate_img(pic2,4),6)
                            elif nx_diff < 0 and py_diff > 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,4)
                                
                        elif ny_diff != 0 and py_diff == 0:
                            if ny_diff < 0 and px_diff < 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,3)
                            elif ny_diff > 0 and px_diff < 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,6)
                            elif ny_diff > 0 and px_diff > 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,1)
                            elif ny_diff < 0 and px_diff > 0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic2,7)
                                                                                             
                        elif ny_diff != 0 and py_diff != 0:
                            if py_diff >0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,4)
                            elif py_diff < 0:                                                                             
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,2)
                                                                                                            
                        elif ny_diff == 0 and py_diff == 0:
                            if px_diff >0:
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,1)
                            elif px_diff < 0:                                                                             
                                self.ans_board_cv2[i*30:i*30+pic_size,j*30:j*30+pic_size] = self.rotate_img(pic1,3)                        
  

    def mousePressEvent(self, event):
        # 1~12 행열좌표를 구함
        x = event.x()-10
        y = event.y()-10
        ty = (x-11)//30 +1
        tx = (y-11)//30 +1      
        try:
            if self.m_df.iloc[(tx-1,ty-1)] == 95 and tx>=1 and tx<=12 and ty>=1 and ty<=12:#이미 열려있는 칸 패스
                if tx >=5 and tx <= 8 and ty >=5 and ty <=8: #꽃 패스
                    pass
                else:
                    if event.buttons() & Qt.LeftButton:
                        self.play(1,tx, ty,self.board_cv2)
                    if event.buttons() & Qt.RightButton:
                        if self.light_dig > 0: # 얕게파기 가능하면
                            self.light_dig -= 1
                            self.play(2,tx,ty,self.board_cv2)
        except:
            pass #오류 패스
  
        
    def play(self,leftOrRight,x,y,board_img): #leftOrRight = 1(left click) /2(right click)
        #input_info
        click_around = lk.input_info.xy(x,y)
        
        #click_event - 표시할 셀 위치 list로 리턴
        ck = lk.click_event(self.ans_df,click_around,self.hoe_level)
        if leftOrRight ==1:
            rt_loc = ck.left_click()
            
        elif leftOrRight ==2:
            rt_loc = ck.right_click()
                    #to_gui
                
        gui = lk.to_gui(self.m_df,rt_loc)
        self.m_df , find_dirt , cnt_broken, cnt_grav, cnt_root = gui.to_df()

        status = round((self.total_cnt_root-cnt_broken)/self.total_cnt_root,2)*100
        left_dirt= self.total_cnt_dirt - find_dirt
        left_grav = self.total_cnt_gravel- cnt_grav
        left_root = self.total_cnt_root-cnt_root
        #print("\n남은 얕게 파기 횟수: {left_light}\n남은 흙: {dirt} \n남은 자갈: {grav} \n뿌리 상태: {stat}% " \
              #    .format(left_light=self.light_dig, dirt=left_dirt,grav=left_grav,stat =status))  
        
        self.label_7.setText(str(left_root))
        self.label_8.setText(str(left_dirt))
        self.label_9.setText(str(left_grav))
        self.label_10.setText(str(self.light_dig))
        self.progressBar.setValue(status)
        if leftOrRight ==2 and self.m_df.iloc[(x-1,y-1)] == 95: #얕게파기결과가 흙이면 정답그림 반환 xx
            pass
        else:
            for i in range(len(rt_loc)):

                step_size = 30

                xx = rt_loc[i][0][0]
                yy= rt_loc[i][0][1]
                x_step = (xx)*step_size
                y_step = (yy)*step_size
                pic_size = 30
                board_img[x_step:x_step+pic_size,y_step:y_step+pic_size] = self.ans_board_cv2[x_step:x_step+pic_size,y_step:y_step+pic_size]

            self.board_cv2 = board_img
            height, width, channel = self.board_cv2.shape
            bytesPerLine = 3 * width
            qImg_board = QImage(self.board_cv2.data, width, height, bytesPerLine, QImage.Format_RGB888)
            self.label_img.setPixmap(QPixmap(qImg_board))  

        if left_dirt == 0: #게임 종료시
            print('game end')
            
            height, width, channel = self.board_cv2.shape
            bytesPerLine = 3 * width
            qImg_board = QImage(self.ans_board_cv2.data, width, height, bytesPerLine, QImage.Format_RGB888)
            self.label_img.setPixmap(QPixmap(qImg_board))   
            self.label_7.setText(str(0))
            self.label_8.setText(str(0))
            self.label_9.setText(str(0))
    
    def rotate_img(self,img,angle=5): #angle 생략시 랜덤
        
        if angle == 5:
        # random rotate image
            rn = rd.randint(1, 4)
        elif angle < 5:
            rn= angle #1~4
        elif angle >5 :
            if angle == 6:
                rn = angle
            elif angle ==7:
                rn =angle
            
        if rn ==1:
            img_0 = img
        elif rn==2:
            img_0 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif rn==3:
            img_0 = cv2.rotate(img, cv2.ROTATE_180)
        elif rn==4:
            img_0 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif rn==6:
            img_0 = cv2.flip(img, 0) #상하반전
        elif rn==7:
            img_0 = cv2.flip(img, 1) #좌우반전
            
        return img_0
        
if __name__ == '__main__':

    app = QApplication(sys.argv)
    ex = MyApp()
    ex.show()
    sys.exit(app.exec_())

     0   1   2   3   4   5   6   7   8   9   10  11
row                                                
0     0   0  91   0   0  47  46   0   0   0  91   0
1    18   0   0   0  49  48  45  91  91   0   0   0
2    17  16   0   0  42  43  44   0   0   0   0   0
3    91  15  91   0  41   0   0   0   0  28   0   0
4     0  14  13   0  99  99  99  99  91  27  26  25
5     0   0  12  11  99  99  99  99  21  22  23  24
6     0  91   0   0  99  99  99  99  91   0   0   0
7     0   0  32  31  99  99  99  99   0   0   0   0
8     0   0  33  65  64  61  91  51   0   0   0   0
9     0   0  34  66  63  62   0  52   0   0  91   0
10    0  91  35  67   0  57  56  53   0   0   0   0
11    0  37  36   0   0  58  55  54   0   0   0   0 47 69 12


<ipython-input-1-9cb3a97eebb0>:251: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.progressBar.setValue(status)


game end
     0   1   2   3   4   5   6   7   8   9   10  11
row                                                
0     0  91   0  91  91   0   0   0   0  91   0   0
1     0   0   0   0   0   0   0   0  91   0   0   0
2    91   0   0   0  22  23  24  91  91   0  91   0
3     0   0   0   0  21  26  25   0   0  91   0   0
4     0   0   0   0  99  99  99  99   0  37  36  35
5    14  13  12  11  99  99  99  99  31  32  33  34
6    15  16  17   0  99  99  99  99   0   0   0   0
7     0  53  52  51  99  99  99  99  91   0   0   0
8    55  54   0   0   0   0   0  41   0   0  91   0
9    56   0   0   0   0  91   0  42  43  44  45   0
10   57   0   0   0   0   0   0  91   0   0  46   0
11   58  59   0   0   0   0   0   0   0   0  47   0 36 78 14
     0   1   2   3   4   5   6   7   8   9   10  11
row                                                
0     0   0   0   0  91   0   0   0   0   0  91   0
1    91  91   0  91  91   0   0   0   0   0   0  59
2     0   0  27  26  25  24  23  22  55  56  5

     0   1   2   3   4   5   6   7   8   9   10  11
row                                                
0     0   0   0   0   0   0   0   0  15  16   0   0
1     0   0   0   0   0  54  53  13  14  17   0   0
2    91   0   0   0   0  55  52  12  91  18  91   0
3     0   0   0   0   0  56  51  11   0   0   0   0
4     0  91  91   0  99  99  99  99   0   0   0   0
5    91   0  46  45  99  99  99  99   0  91   0   0
6    91   0  43  44  99  99  99  99   0   0   0   0
7    91   0  42  41  99  99  99  99  31  32   0   0
8     0   0   0   0  25  24  21   0   0  33  36   0
9     0   0   0  91  26  23  22   0  91  34  35   0
10    0  91   0  91  27  28  91   0   0   0   0   0
11    0   0   0   0   0   0   0   0   0   0   0   0 34 80 14


<ipython-input-1-9cb3a97eebb0>:251: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.progressBar.setValue(status)


SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
